In [1]:
import enum
import torch
import torch.nn as nn
from options import Option
from dataset import create_loader
from collections import OrderedDict
from models.quant_ops import q_k, QConv2d, set_option
from torch.utils.tensorboard import SummaryWriter
from log_utils import *

import pandas as pd
import pickle

In [2]:
option = Option("./cifar100.hocon", "test")
# 기존에 있는 log 를 불러오기 위해 log_override를 false로 하고 진행
option.log_override=False
option.set_save_path()

/data/adabits/save_log/cifar100_resnet_cifar/log_cifar100_resnet_cifar_bs128_ep200_seed_3/ is exists
load log path /data/adabits/save_log/cifar100_resnet_cifar/log_cifar100_resnet_cifar_bs128_ep200_seed_3/


In [3]:
import models
model_names = sorted(name for name in models.__dict__
                     if name.islower() and not name.startswith("__")
                     and callable(models.__dict__[name]))


In [4]:
callable(models.__dict__["quant_ops"])

False

In [12]:
writer = SummaryWriter()
test_dict = {"a":1, "b":2, "c":3, "aa":5}
sub_dict = {key:value for key, value in test_dict.items() if "a" in key}

In [15]:
sub_dict.update({"s":5})

In [16]:
sub_dict

{'a': 1, 'aa': 5, 's': 5}

AttributeError: type object 'Model' has no attribute 'OPTION'

In [3]:
class Compare_BatchNorm_Precision(nn.Module):
    # this is custom batchnorm different precision
    # Considered https://github.com/ptrblck/pytorch_misc/blob/master/batch_norm_manual.py and rangeBN 

    def __init__(self, num_features, dim=1, momentum=0.9, affine=True, eps=1e-5, compute_type=torch.float16):
        super(Compare_BatchNorm_Precision, self).__init__()
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.zeros(num_features))

        self.momentum = momentum
        self.dim = dim
        self.eps = 1e-10
        if affine:
            self.bias = nn.Parameter(torch.Tensor(num_features))
            self.weight = nn.Parameter(torch.Tensor(num_features))
        self.compute_type = compute_type
        self.eps = eps
        self.reset_params()

    def reset_params(self):
        if self.weight is not None:
            self.weight.data.uniform_()
        if self.bias is not None:
            self.bias.data.zero_()

    def load_params(self, weight, bias, running_mean, running_var):
        self.weight=weight
        self.bias=bias
        self.running_mean = running_mean
        self.running_var = running_var
        

    def forward(self, x, inference=True):
        x_ch = x.type(self.compute_type)        
        if inference:
            mean = self.running_mean
            scale = self.running_var
            mean_ch = self.running_mean.type(self.compute_type)
            scale_ch = self.running_var.type(self.compute_type)

            out = (x - mean.view(1, mean.size(0), 1, 1)) * \
                scale.view(1, scale.size(0), 1, 1)

            out_ch = (x_ch - mean_ch.view(1, mean_ch.size(0), 1, 1)) * \
                scale_ch.view(1, scale_ch.size(0), 1, 1)


        else:
            c = x_ch.shape[1]
            mean = x.transpose(0,1).reshape(c, -1).mean(dim=-1)
            scale = x.transpose(0,1).reshape(c, -1).var(dim=-1, unbiased=False)
            mean_ch = x_ch.transpose(0,1).reshape(c, -1).mean(dim=-1)
            scale_ch = x_ch.transpose(0,1).reshape(c, -1).var(dim=-1, unbiased=False)

            out = (x - mean.view(1, mean.size(0), 1, 1)) * \
                torch.sqrt(scale.view(1, scale.size(0), 1, 1) + self.eps)

            out_ch = (x_ch - mean_ch.view(1, mean_ch.size(0), 1, 1)) * \
                torch.sqrt(scale_ch.view(1, scale.size(0), 1, 1) + self.eps.type(self.compute_type))
        
        if self.weight is not None:
            
            weight = self.weight
            weight_ch = self.weight.type(self.compute_type)
            out = out * weight.view(1, weight.size(0), 1, 1)
            out_ch = out_ch * weight_ch.view(1, weight_ch.size(0), 1, 1)

        if self.bias is not None:
            bias =self.bias
            out = out * bias.view(1, bias.size(0), 1, 1)
            bias_ch = self.bias.type(self.compute_type)
            out_ch = out_ch * bias_ch.view(1, bias_ch.size(0), 1, 1)

        return out, out_ch